# INSTALAR LIBRERIAS

In [13]:
# !pip install docx2pdf
#pip install ipywidgets

# IMPORTAR

In [1]:
from docx2pdf import convert
import os
import time

# CONSTRUIR LA RUTA DE LOS ARCHIVOS

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../../../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [3]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
# colocar aqui la ruta de los archivos
path = ruta_completa + r'/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 11 INFORMES
11 INFORMES


['C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_2641_CDSE_19-03-2025_CRONENBOLD AGUILERA ROBERTO_183_L45.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_2641_CDSE_19-03-2025_CRONENBOLD AGUILERA ROBERTO_184_L1.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_2662_CDSE_18-03-2025_DON SERAFIN SRL._179_L28.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_2662_CDSE_18-03-2025_DON SERAFIN SRL._180_L14.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_30035_CDSE_20-03-2025_GUTIERREZ AGUILERA WIDEN GUILLER_185_L2.2.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_30035_CDSE_20-03-2025_GUTIERREZ AGUILERA WIDEN GUILLER_186_L6.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_30035_CDSE_20-03-2025_GUTIERREZ AGUILERA WIDEN

# INICIA EL PROCESO DE CONVERCIÓN

In [13]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/1530_CRS_08-03-2025_C.I.T.T.C.A._566.pdf'

In [14]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/1530_CRS_08-03-2025_C.I.T.T.C.A._566.pdf'

In [16]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\bismarksr\Downloads\\' + pdf)
    i+=1
    time.sleep(5)
print('END')

CONVIRTIENDO... INFORME 1 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/13062_CRS_06-03-2025_SOTO ARCE BACILIO_483.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/13062_CRS_06-03-2025_SOTO ARCE BACILIO_483.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/13062_CRS_06-03-2025_SOTO ARCE BACILIO_483.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/1530_CRS_08-03-2025_C.I.T.T.C.A._566.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/1530_CRS_08-03-2025_C.I.T.T.C.A._566.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/1530_CRS_08-03-2025_C.I.T.T.C.A._566.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/31088_CRS_08-03-2025_COLQUE QUIROZ JENNY FELICIDAD_523.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/31088_CRS_08-03-2025_COLQUE QUIROZ JENNY FELICIDAD_523.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/31088_CRS_08-03-2025_COLQUE QUIROZ JENNY FELICIDAD_523.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 4 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/41533_CRS_08-03-2025_ARNEZ COLQUE ROSARIO ESTELA_524.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/41533_CRS_08-03-2025_ARNEZ COLQUE ROSARIO ESTELA_524.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/41533_CRS_08-03-2025_ARNEZ COLQUE ROSARIO ESTELA_524.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


# Leer archivos

In [1]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [6]:
contenido = os.listdir(ruta_completa + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

ValueError: invalid literal for int() with base 10: 'inf'

# CARGAR LOS PDFS A GOOGLE DRIVE

In [21]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta_completa + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 13062_CRS_06-03-2025_SOTO ARCE BACILIO_483.pdf with mimeType application/pdf
Created file 1530_CRS_08-03-2025_C.I.T.T.C.A._566.pdf with mimeType application/pdf
Created file 31088_CRS_08-03-2025_COLQUE QUIROZ JENNY FELICIDAD_523.pdf with mimeType application/pdf
Created file 41533_CRS_08-03-2025_ARNEZ COLQUE ROSARIO ESTELA_524.pdf with mimeType application/pdf
end


In [22]:
print(len(dic_urls), 'URLs')
dic_urls

4 URLs


{13062: 'https://drive.google.com/file/d/1sntgEm4boAO312yusmUCsVmxbzPWkNx5/view?usp=drivesdk',
 1530: 'https://drive.google.com/file/d/1o4IH-UbmWD9PlqLJroWLxc1ZxchonqeW/view?usp=drivesdk',
 31088: 'https://drive.google.com/file/d/1x4OlhIsGu4_UwKqtH1WPSsCzDrI6k8R-/view?usp=drivesdk',
 41533: 'https://drive.google.com/file/d/1Bsc1iYRtEkc3PG9uepUCgryIkCYKBEdn/view?usp=drivesdk'}

# conexion a amigocloud

In [23]:
import sys
sys.path.append('../../../_amigocloud')

from amigocloud import AmigoCloud

In [24]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [25]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

13062
1530
31088
41533
end
